In [1]:
import pandas as pd 
import os 

from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

import seaborn as sns

from datasets import load_dataset, load_from_disk
import numpy as np
import json

In [2]:
pwd

'/mnt/home/al2644/research/projects/perturb-r/notebook'

In [5]:
cd /mnt/home/al2644/research/projects/rlvr

/mnt/home/al2644/research/projects/perturb-r


In [6]:
from utils import chunk_r 

# Check the Dataset

In [13]:
path = "data/countdown_train_stage2_level5_100K/"
dataset = load_from_disk(path)

In [16]:
dataset['test'][0]

{'nums': [61, 11, 50, 60, 71],
 'target': 133,
 'example_solution': '(((71+61)+50)-(60-11))',
 'problem': 'Please reason and answer the following question. Using the numbers [61, 11, 50, 60, 71], create an equation that equals 133. You can only use basic arithmetic operations (+, -, *, /) in the expression and each number should be used exactly once. You should report the answer equation expression in <answer> </answer> tags, for example <answer> (1 + 2) / 3 </answer>.',
 'level': 5,
 'task': 'countdown'}

# Training Data

In [18]:
cd /mnt/home/al2644/research/projects/rlvr

/mnt/home/al2644/research/projects/rlvr


In [21]:
stage1_df = pd.read_parquet("data/train/countdown_level5_35k/test.parquet")
stage2_df = pd.read_parquet("data/train/countdown_level5_50k/test.parquet")

In [22]:
stage1_df

,data_source,prompt,ability,reward_model,extra_info
0,countdown,[{'content': 'Please reason and answer the fol...,math,"{'ground_truth': {'numbers': [56, 46, 14, 36, ...","{'index': 0, 'level': '6', 'split': 'test'}"
1,countdown,[{'content': 'Please reason and answer the fol...,math,"{'ground_truth': {'numbers': [2, 76, 58, 8, 53...","{'index': 1, 'level': '6', 'split': 'test'}"
2,countdown,[{'content': 'Please reason and answer the fol...,math,"{'ground_truth': {'numbers': [32, 12, 23, 86, ...","{'index': 2, 'level': '6', 'split': 'test'}"
3,countdown,[{'content': 'Please reason and answer the fol...,math,"{'ground_truth': {'numbers': [6, 17, 92, 23, 7...","{'index': 3, 'level': '6', 'split': 'test'}"
4,countdown,[{'content': 'Please reason and answer the fol...,math,"{'ground_truth': {'numbers': [23, 43, 33, 33, ...","{'index': 4, 'level': '6', 'split': 'test'}"
...,...,...,...,...,...
295,countdown,[{'content': 'Please reason and answer the fol...,math,"{'ground_truth': {'numbers': [95, 38, 13, 72, ...","{'index': 295, 'level': '7', 'split': 'test'}"
296,countdown,[{'content': 'Please reason and answer the fol...,math,"{'ground_truth': {'numbers': [51, 100, 99, 91,...","{'index': 296, 'level': '7', 'split': 'test'}"
297,countdown,[{'content': 'Please reason and answer the fol...,math,"{'ground_truth': {'numbers': [44, 2, 3, 41, 85...","{'index': 297, 'level': '7', 'split': 'test'}"
298,countdown,[{'content': 'Please reason and answer the fol...,math,"{'ground_truth': {'numbers': [55, 20, 64, 3, 4...","{'index': 298, 'level': '7', 'split': 'test'}"


In [23]:
stage2_df

,data_source,prompt,ability,reward_model,extra_info
0,countdown,[{'content': 'Please reason and answer the fol...,math,"{'ground_truth': {'numbers': [50, 77, 64, 67, ...","{'index': 0, 'level': '6', 'split': 'test'}"
1,countdown,[{'content': 'Please reason and answer the fol...,math,"{'ground_truth': {'numbers': [88, 84, 11, 71, ...","{'index': 1, 'level': '6', 'split': 'test'}"
2,countdown,[{'content': 'Please reason and answer the fol...,math,"{'ground_truth': {'numbers': [35, 48, 94, 46, ...","{'index': 2, 'level': '6', 'split': 'test'}"
3,countdown,[{'content': 'Please reason and answer the fol...,math,"{'ground_truth': {'numbers': [18, 58, 64, 42, ...","{'index': 3, 'level': '6', 'split': 'test'}"
4,countdown,[{'content': 'Please reason and answer the fol...,math,"{'ground_truth': {'numbers': [37, 18, 51, 39, ...","{'index': 4, 'level': '6', 'split': 'test'}"
...,...,...,...,...,...
295,countdown,[{'content': 'Please reason and answer the fol...,math,"{'ground_truth': {'numbers': [95, 38, 13, 72, ...","{'index': 295, 'level': '7', 'split': 'test'}"
296,countdown,[{'content': 'Please reason and answer the fol...,math,"{'ground_truth': {'numbers': [51, 100, 99, 91,...","{'index': 296, 'level': '7', 'split': 'test'}"
297,countdown,[{'content': 'Please reason and answer the fol...,math,"{'ground_truth': {'numbers': [44, 2, 3, 41, 85...","{'index': 297, 'level': '7', 'split': 'test'}"
298,countdown,[{'content': 'Please reason and answer the fol...,math,"{'ground_truth': {'numbers': [55, 20, 64, 3, 4...","{'index': 298, 'level': '7', 'split': 'test'}"


# Corruption Data

In [89]:
cd ../perturb-r/

/mnt/home/al2644/research/projects/perturb-r


In [90]:
path = "./results/countdown_train_stage2_level5_100K/corrupt_numbers/Qwen2.5-3B-countdown-level4-5-stage1_rl.pickle"
df = pd.read_pickle(path)

In [112]:
df.columns

Index(['nums', 'target', 'example_solution', 'problem', 'level', 'task',
       'original_response', 'reasoning_chunks', 'start_pos', 'end_pos',
       'prefix_reasoning', 'corrupted_reasoning', 'replacement', 'prompt',
       'post_corruption_response', 'still_correct', 'if_answer'],
      dtype='object')

In [111]:
df[df['still_correct']==1.]["prompt"].iloc[1500]

"<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\nPlease reason and answer the following question. Using the numbers [77, 67, 5, 61, 40], create an equation that equals 217. You can only use basic arithmetic operations (+, -, *, /) in the expression and each number should be used exactly once. You should report the answer equation expression in <answer> </answer> tags, for example <answer> (1 + 2) / 3 </answer>.<|im_end|>\n<|im_start|>assistant\n<think>\nOkay, let's see. I need to use the numbers 77, 67, 5, 61, and 40 exactly once each, and combine them with basic operations to get 217. Hmm. Let me start by thinking about possible combinations. \n\nFirst, maybe I should check if adding some numbers and subtracting others would work. Let me list the numbers again: 77, 67, 5, 61, 40. The target is 217. Let's consider the total sum of all numbers. Adding them up: 77 + 67 is 144, plus 5 is 149, plus 61 is 210, plus 40 is 250. Oh, that's more\n\n\nthan 217. So m

In [72]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-3B")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [74]:
wrong_df = df[df["still_correct"]==0.]
wrong_df["prompt_token_count"]=wrong_df["prompt"].apply(lambda x: len(tokenizer.encode(x)))

/tmp/ipykernel_1020141/4048595744.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wrong_df["prompt_token_count"]=wrong_df["prompt"].apply(lambda x: len(tokenizer.encode(x)))


In [78]:
wrong_df[wrong_df["prompt_token_count"]<=2048]

,nums,target,example_solution,problem,level,task,original_response,reasoning_chunks,start_pos,end_pos,prefix_reasoning,corrupted_reasoning,replacement,prompt,post_corruption_response,still_correct,if_answer,prompt_token_count
5,"[37, 87, 53, 13, 95]",834,((((13*53)+95)-37)+87),Please reason and answer the following questio...,5,countdown,"<think>\nOkay, let's try to figure out how to ...","[<think>\nOkay, let's try to figure out how to...",0.00,0.26,,"<think>\nOkay, let's try to figure out how to ...","{'4.56': '45.6', '834': '438', '9': '4', '9.5'...",<|im_start|>system\nYou are a helpful assistan...,"Alternatively, 31 + something?\n\nAlternative...",0.0,True,531
7,"[37, 87, 53, 13, 95]",834,((((13*53)+95)-37)+87),Please reason and answer the following questio...,5,countdown,"<think>\nOkay, let's try to figure out how to ...","[<think>\nOkay, let's try to figure out how to...",0.26,0.53,"<think>\nOkay, let's try to figure out how to ...","Alternatively, maybe 533 * 1 = 69. Then 7 * so...","{'11.2': '1.12', '983': '83', '7600': '7006', ...",<|im_start|>system\nYou are a helpful assistan...,Not helpful.\n\nAnother angle. 95 + 87 is 182...,0.0,True,932
19,"[6, 66, 50, 69, 80]",259,((((69-6)+66)+80)+50),Please reason and answer the following questio...,5,countdown,"<think>\nOkay, let's see. I need to use the nu...","[<think>\nOkay, let's see. I need to use the n...",0.00,0.27,,"<think>\nOkay, let's see. I need to use the nu...","{'69': '9', '116': '1116', '265': '526', '6': ...",<|im_start|>system\nYou are a helpful assistan...,"check that. \n\nWait, 5 is missing. Did I list...",0.0,True,331
20,"[10, 53, 89, 61, 13]",991,(((89*10)+(61+53))-13),Please reason and answer the following questio...,5,countdown,"<think>\nOkay, let's see. I need to use the nu...","[<think>\nOkay, let's see. I need to use the n...",0.00,0.31,,"<think>\nOkay, let's see. I need to use the nu...","{'155': '1155', '689': '6889', '610': '601', '...",<|im_start|>system\nYou are a helpful assistan...,"9449. Not enough. Hmm, this seems too vague. W...",0.0,False,478
22,"[10, 53, 89, 61, 13]",991,(((89*10)+(61+53))-13),Please reason and answer the following questio...,5,countdown,"<think>\nOkay, let's see. I need to use the nu...","[<think>\nOkay, let's see. I need to use the n...",0.00,0.31,,"<think>\nOkay, let's see. I need to use the nu...","{'155': '15', '689': '68', '610': '6610', '839...",<|im_start|>system\nYou are a helpful assistan...,"6610? No, maybe that's not helpful. Let me th...",0.0,True,458
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24981,"[39, 59, 82, 27, 23]",12,(((23-27)-(82-39))+59),Please reason and answer the following questio...,5,countdown,"<think>\nOkay, let's see. I need to use the nu...","[<think>\nOkay, let's see. I need to use the n...",0.00,0.27,,"<think>\nOkay, let's see. I need to use the nu...","{'27': '2', '50': '5', '1.444': '14.44', '39':...",<|im_start|>system\nYou are a helpful assistan...,Maybe this approach is not working.\n\nAltern...,0.0,False,382
24986,"[46, 3, 1, 72, 11]",34,((72-3)-((1*46)-11)),Please reason and answer the following questio...,5,countdown,"<think>\nOkay, let's see. I need to use the nu...","[<think>\nOkay, let's see. I need to use the n...",0.00,0.27,,"<think>\nOkay, let's see. I need to use the nu...","{'40': '4', '58': '558', '1': '8', '29': '2', ...",<|im_start|>system\nYou are a helpful assistan...,"Alternatively, 111 minus something? 111 - 8 is...",0.0,False,537
24990,"[35, 1, 33, 31, 49]",77,(((49-1)+(33-35))+31),Please reason and answer the following questio...,5,countdown,"<think>\nOkay, let's see. I need to use the nu...","[<think>\nOkay, let's see. I need to use the n...",0.00,0.27,,"<think>\nOkay, let's see. I need to use the nu...","{'31': '331', '100': '10', '86': '6', '1': '4'...",<|im_start|>system\nYou are a helpful assistan...,"For example, maybe (something) - (something e...",0.0,True,395
24992,"[35, 1, 33, 31, 49]",77,(((49-1)+(33-35))+31),Please reaso

In [95]:
cd /mnt/home/al2644/research/projects/rlvr/

/mnt/home/al2644/research/projects/rlvr


In [96]:
train_data = pd.read_parquet("./data/train/countdown_level5_corrupt_numbers_14k/")

In [102]:
train_data["prompt"][13483]

array([{'content': 'Please reason and answer the following question. Using the numbers [39, 59, 82, 27, 23], create an equation that equals 12. You can only use basic arithmetic operations (+, -, *, /) in the expression and each number should be used exactly once. You should report the answer equation expression in <answer> </answer> tags, for example <answer> (1 + 2) / 3 </answer>.', 'role': 'user'},
       {'content': "<think>\nOkay, let's see. I need to use the numbers 9, 9, 28, 2, and 233 exactly once each, with basic operations to make 2. Hmm. Let me think about possible combinations.\n\nFirst, maybe I can try subtracting some numbers. For example, if I take 28 minus something. Let's see. 28 minus 9 is 233. Wait, but then I already have a 233. Oh, maybe that's not helpful. Alternatively, maybe 9 minus something? Let's try different approaches.\n\nAnother thought: maybe combining numbers through division or multiplication. For instance, 9 divided by something? Let's see. 9 divided 

In [113]:
df = pd.read_pickle("/mnt/home/al2644/research/projects/perturb-r/results/countdown_stage2/inject_thoughts/Qwen2.5-3B-countdown-level4-5-stage1_rl.pickle")

In [114]:
df

,nums,target,example_solution,problem,level,task,original_response,correct,prompt,inject_type,post_inject_response,still_correct,if_answer
0,"[22, 13, 86, 32, 42]",350,(((32*13)+42)-(86+22)),Please reason and answer the following questio...,5,countdown,"<think>\nOkay, let's see. I need to use the nu...",1.0,<|im_start|>system\nYou are a helpful assistan...,half,"verting the equation.\n\nAlternatively, let's ...",1.0,True
1,"[31, 71, 90, 15, 1]",3,((((15-71)-31)*1)+90),Please reason and answer the following questio...,5,countdown,"<think>\nOkay, so I need to use the numbers 31...",1.0,<|im_start|>system\nYou are a helpful assistan...,half,"ithiness of the problem, perhaps there is anot...",0.0,True
2,"[88, 37, 52, 52, 13]",853,(((13*52)+52)+(37+88)),Please reason and answer the following questio...,5,countdown,"<think>\nOkay, let's see. I need to use the nu...",1.0,<|im_start|>system\nYou are a helpful assistan...,full,"Physical, Undetected, Blanking, Guerillas, He...",0.0,True
3,"[48, 67, 10, 67, 37]",21,(((67+48)+(10-67))-37),Please reason and answer the following questio...,5,countdown,"<think>\nOkay, let's see. I need to use the nu...",1.0,<|im_start|>system\nYou are a helpful assistan...,half,"\nOkay, continuing my attempt. Let me plan how...",1.0,True
4,"[31, 9, 63, 53, 17]",300,((((17*9)+53)+31)+63),Please reason and answer the following questio...,5,countdown,"<think>\nOkay, let's see. I need to use the nu...",1.0,<|im_start|>system\nYou are a helpful assistan...,full,) multiplied by 10. But the problem states tha...,1.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,"[43, 89, 52, 22, 24]",182,((52+43)-((24-22)-89)),Please reason and answer the following questio...,5,countdown,"<think>\nOkay, let's see. I need to use the nu...",1.0,<|im_start|>system\nYou are a helpful assistan...,half,"3 equations are given. Let me see, the problem...",1.0,True
49996,"[34, 58, 53, 81, 98]",94,((58-34)+((98-81)+53)),Please reason and answer the following questio...,5,countdown,"<think>\nOkay, let's see. I need to use the nu...",1.0,<|im_start|>system\nYou are a helpful assistan...,half,"ulibria"", but in the previous hint there's one...",1.0,True
49997,"[52, 33, 41, 7, 57]",495,(((57+41)+33)+(7*52)),Please reason and answer the following questio...,5,countdown,"<think>\nOkay, let's see. I need to use the nu...",1.0,<|im_start|>system\nYou are a helpful assistan...,half,"ter supposed to use all the numbers once, and ...",1.0,True
49998,"[34, 15, 76, 13, 62]",102,((((62-34)+13)-15)+76),Please reason and answer the following questio...,5,countdown,"<think>\nOkay, let's see. I need to use the nu...",1.0,<|im_start|>system\nYou are a helpful assistan...,full,"', y', z') where x', y', z' are on the surface...",1.0,True
